# Lab 1 - Naive Bayes Classifier

## Work breakdown
* Taras Rodzin - `fit()` and its helper functions
* Taras Onyshkiv - `process_data()` and its helper functions
* Olesia Nedopas - `predict_prob()` , `predict()` and `score()`

The descriptive contributions to this notebook, function documentation and construction of the algorithm were made jointly

## Introduction
During the past three weeks, you learned a couple of essential notions ant theorems. One of them is Bayes theorem.

One of its applications is **Naive Bayes classifier**, which is a probabilistic classifier whose aim is to determine which class some observation probably belongs by using the Bayes formula:
$$\mathsf{P}(\mathrm{class}\mid \mathrm{observation})=\frac{\mathsf{P}(\mathrm{observation}\mid\mathrm{class})\mathsf{P}(\mathrm{class})}{\mathsf{P}(\mathrm{observation})}$$

Under the strong independence assumption, one can calculate $\mathsf{P}(\mathrm{observation} \mid \mathrm{class})$ as
$$\mathsf{P}(\mathrm{observation}) = \prod_{i=1}^{n} \mathsf{P}(\mathrm{feature}_i),$$
where $n$ is the total number of features describing a given observation. Thus, $\mathsf{P}(\mathrm{class}|\mathrm{observation})$ now can be calculated as

$$\mathsf{P}(\mathrm{class} \mid \mathrm{\mathrm{observation}}) = \mathsf{P}(\mathrm{class})\times \prod_{i=1}^{n}\frac{\mathsf{P}(\mathrm{feature}_i\mid \mathrm{class})}{\mathsf{P}(\mathrm{feature}_i)}$$

For more detailed explanation, you can check [this link](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/).



## Data  description


* **sentiment**
All the text messages contained in this data set are labeled with three sentiments: positive, neutral or negative. The task is to classify some text message as the one of positive mood, negative or neutral.



##Implementation
Processing the data

In [ ]:
import pandas as pd
import string


def find_stop_words(path: str) -> list:
    """
    Generates stop words to reduce the messages' space
    """
    with open(path, "r") as file:
        result = [line.replace("\n", "") for line in file]
    return result


def remove_extra(text: str) -> str:
    """
    Removes all punctuation from message
    :param: str - Tito, mutti
    :return: str - Tito mutti
    """
    stop_words = find_stop_words("/stop_words.txt")
    text_list = text.split(" ")

    for word in text_list:

        if word in stop_words:
            text_list.remove(word)

    text = " ".join(text_list)

    return text.translate(str.maketrans('', '', string.punctuation))


def gen_box_of_words(message: list) -> dict:
    """
    Convert message to a box of words
    :param message: list of words
    """
    word_box = {}

    for word in message:

        if word not in word_box:
            word_box[word] = 1
        else:
            word_box[word] += 1

    return word_box


def prepare_message(message: str) -> dict:
    """
    :param message: str - message preparation to be converted
    """
    message = message.lower()
    message = remove_extra(message)
    message = list(filter(lambda x: x != "", message.split(' ')))

    return gen_box_of_words(message)


def process_data(data_file: str) -> tuple:
    """
    Function for data processing and split it into X and y sets.
    :param data_file: str - train datado a research of your own
    :return: pd.DataFrame|list, pd.DataFrame|list - X and y data frames or lists
    """
    df = pd.read_csv(data_file)
    df['text'] = df['text'].apply(prepare_message)

    return (list(df["text"]), list(df["sentiment"]))

### Implementation
The Naive Bayes Classifier 


In [ ]:
class BayesianClassifier:
    """
    Implementation of Naive Bayes classification algorithm.
    """

    def __init__(self):
        self.partition_sizes = None
        self.partition_probs = None
        self.partition_word_sizes = None
        self.messages = None
        self.labels = None
        self.words = None

    def fit(self, X, y):
        """
        Fit Naive Bayes parameters according to train data X and y.
        :param X: pd.DataFrame|list - train input/messages
        :param y: pd.DataFrame|list - train output/labels
        :return: None
        """
        self.messages = X
        self.labels = y
        self.words = calculate_uniques(self.messages)

        self.partition_sizes, self.partition_word_sizes = calculate_partitions(
            self.labels, self.messages)

        self.partition_probs = calculate_partition_probs(
            self.partition_sizes, len(self.labels))

    def predict_prob(self, message, label):
        """
        Calculate the probability that a given label can be assigned to a given message.
        :param message: str - input message
        :param label: str - label
        :return: float - probability P(label|message)
        """
        prob = self.partition_probs[label]

        for word in message:

            word_in_partition = word_given_partition(
                word, self.messages, self.labels, label)

            prob *= word_in_partition / \
                (self.partition_word_sizes[label] + len(self.words))

        return prob

    def predict(self, message):
        """
        Predict label for a given message.
        :param message: str - message
        :return: str - label that is most likely to be truly assigned to a given message
        """
        partition_probabilities = {}

        for label in self.partition_sizes:
            partition_probabilities[self.predict_prob(message, label)] = label

        return partition_probabilities[max(partition_probabilities)]

    def score(self, X, y):
        """
        Return the mean accuracy on the given test data and labels - the efficiency of a trained model.
        :param X: pd.DataFrame|list - test data - messages
        :param y: pd.DataFrame|list - test labels
        :return:
        """
        score = 0

        for idx, message in enumerate(X):
            result = self.predict(message)

            if result == y[idx]:
                score += 1

        return score/len(X)


def calculate_partitions(labels: list, messages: list) -> dict:
    """
    Return number of messages and words in each partition
    :param labels: list of labels
    :param messages: list of messages
    """
    sentiments = {
        "positive": 0,
        "neutral": 0,
        "negative": 0
    }

    words_amount = {
        "positive": 0,
        "neutral": 0,
        "negative": 0
    }

    for idx, message in enumerate(messages):

        sentiments[labels[idx]] += 1
        words_amount[labels[idx]] += len(message)

    return sentiments, words_amount


def calculate_partition_probs(partitions: dict, size: int) -> dict:
    """
    Return probabilities of each partition to occur
    :param partitions: dictionary
    :param size: int
    """
    partitions_probs = {}

    for partition in partitions:
        partitions_probs[partition] = partitions[partition] / size

    return partitions_probs


def word_given_partition(word: str, messages: list, labels: list, label: str) -> int:
    """
    Return number of how many times word occurs in partition
    :param word: str
    :param messages: list
    :param labels: list of labels
    :param label: str
    """
    word_partition = 1

    for idx, message in enumerate(messages):
        if word in message and labels[idx] == label:

            word_partition += 1

    return word_partition


def calculate_uniques(messages: list) -> list:
    """
    Return list of unique words
    :param messages: list of messages
    """
    uniques = []

    for message in messages:
        for word in message:
            if word not in uniques:
                uniques.append(word)

    return uniques

### Testing

In [ ]:
train_X, train_y = process_data("/train.csv")
test_X, test_y = process_data("/test.csv")

classifier = BayesianClassifier()
classifier.fit(train_X, train_y)
classifier.predict_prob(test_X[0], test_y[0])

print("model score: ", classifier.score(test_X, test_y))

model score:  0.3223234624145786


## Conclusions





### <b>The method implemented in general:</b>

In this assignment we were tasked with implementing a model, which would predict the sentiment of a sentence, based on the analysis of the training data given to us. These type of problems have to do with conditional probability, so one of the ways to solve them is using the Naive Bayes classifier, which is based on the Bayes theorem. <br> 
Predicting the sentiments of sentences is basically classifying them into 3 categories. To do this, we need to compare the probabilities of a sentence belonging to each of these 3 classes and choosing the most likely one. Using the classifier we calculated the conditional probability that one of the classes can be applied to a sentence by splitting it into words (separate features) and multiplying the probabiities of each of them appearing in a certain class (multiplying this product by the probability of getting the class itself). By analyzing the training data we trained the model to predict, which sentiment, or label, to asssign to a new sentence. <br>
When testing the model on unknown examples, though, there may be some words which are absent in the "bag of words" (the distribution of instances of all words in the training data), in which case the classifier wil assign a probability of 0 to the word, making the sentence impossible to classify. To prevent this, there's a technique called "smoothing", which implies adding 1 to the total instances of every word a priori, and then adding the number of words in the "bag" to the denominator of the probability fraction

### <b>Pros and cons of the method:</b>

#### Pros
* Works fast and efficiently, assuming that the features are independent
* Can solve problems that require classifying features into multiple categories
* Requires less training data than other models to make predictions about the test data
* Easy to implement

#### Cons
* If the test data contains a feature that was absent in the training data, the classifier will not be able to make any predictions about it, because it will assign 0 to the feature
* To make predictions easier it makes an assumption about the features being independent, which may be false in reality, therefore making predictions less accurate. (this is what makes the classifier "naive")
* Suppose our model is trained on data in which the number of elements of a certain partition is much smaller than the number of elements of other partitions. In that case, this model may give the wrong result for sentences from this partition.

### <b>A few sentences about our implementation of the classifier:</b>
* In our implementation of this model on the python, we present all messages as bags of words, which simplifies our further work. We do not represent word bags as a list of ones and zeros, but as dictionaries with elements.
*in the "fit ()" function we generate all the necessary parameters that we will use when searching for probabilities.
*Using the function "predict_prob ()" we find the probability that this message belongs to a certain sentiment by the formula:</br>
        P (sentiment | Message) = P (sentiment) * P (word_1 | sentiment) * (...) * P (word_n | sentiment)
  But to avoid cases where the probability of a word is zero, we initially assume that each word was in at least one message from a given mood. Through we divide the number of cases when you found the word in the message not only on the number of messages but on "the number of messages + the number of unique words."
*In the "predict ()" function, we output the sentiment that is most likely for this message. To do this, we find the probability that this sentence belongs to each of the sentiments and return the sentiment that is most likely.
*To increase the accuracy of our model, we remove all punctuation marks and stop words.

###<b>Describe your results:</b>
If we create a model and run the fit () function based on train.csv and execute the score () function on the data from the test.csv file, we can see that our accuracy is approximately 0.32. From this, we could conclude that we did something wrong. But the problem is not in our implementation of this model but in the databases we use. The problem is that in "train.csv" only 116 of 3968 messages are negative, so the model shows that the probability that the sentence is negative is very small. And in our "test.csv" most of the sentences are negative and our model predicts most of them incorrectly.
